# Evaluation for separate datasets

To answer the referee's request "to split the analysis into the different data sources and report results. That would give the reader an impression how strongly results vary for different (sub-) datasets." 

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import xlsxwriter
from collections import OrderedDict, Counter

import matplotlib.pyplot as plt
plt.style.use('ggplot')

import seaborn as sns
sns.set(color_codes=True)

from evaluators import *
from evaluator import *
from config  import DIR_PATH

%matplotlib inline
%pylab inline

REGISTERED_EVALUATORS = [GenderAPIEvaluator, GenderAPIFullEvaluator, \
    NameAPIEvaluator, NameAPIFullEvaluator, GenderGuesserEvaluator, \
    GenderizeIoEvaluator, NamSorEvaluator]

Populating the interactive namespace from numpy and matplotlib


/Users/helena.mihaljevic-br/.virtualenvs/name-based-inference/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shuffle']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
gender_evalautor_to_service_name = {'gender_api': 'Gender API', 'gender_guesser': 'gender-guesser', 
                                    'genderize_io': 'genderize.io', 'name_api_full': 'NameAPI', 'namsor': 'NamSor'}
def reduce_table(df, by_index):
    """Filter rows or columns of a benchmark table and rename."""
    df = df.sort_index()
    if by_index:
        df = df.loc[gender_evalautor_to_service_name.keys()]
        df = df.rename(index=gender_evalautor_to_service_name)
    else:
        df = df[list(gender_evalautor_to_service_name.keys())]
        df.columns = gender_evalautor_to_service_name.values()
        sorted_cols = sorted(df.columns)
        df = df[sorted_cols]
    return df

## Compute errors without tuning

In [4]:
def compute_all_errors_without_tuning(origin=None):
    service_to_all_errors = {}
    error_names = ['errorCoded', 'errorCodedWithoutNA', 'errorGenderBias', 'naCoded', 'WeightedError']
    
    for s in REGISTERED_EVALUATORS:  
        evaluator = s('all')
        eval_name = evaluator.gender_evaluator
        evaluator.load_data(evaluated=True)
        if origin:
            evaluator.test_data = evaluator.test_data[evaluator.test_data.origin==origin].reset_index(drop=True)
        evaluator._translate_api_response()
        evaluator.compute_confusion_matrix(evaluator.test_data)
        errors = evaluator.compute_all_errors()
        service_to_all_errors[eval_name] = errors
    all_errors = pd.DataFrame.from_dict(service_to_all_errors, orient='index')
    all_errors.columns = error_names
    return all_errors

In [5]:
# Benchmark 1 for the whole dataset
cm = sns.light_palette("green", as_cmap=True)
df = compute_all_errors_without_tuning()
df = reduce_table(df, by_index=True)
df = df.round(3)

df.style.background_gradient(cmap=cm)

,errorCoded,errorCodedWithoutNA,errorGenderBias,naCoded,WeightedError
NameAPI,0.179,0.034,0.004,0.15,0.067
NamSor,0.128,0.043,0.007,0.089,0.061
gender-guesser,0.222,0.026,0.002,0.201,0.073
Gender API,0.079,0.05,-0.011,0.03,0.056
genderize.io,0.143,0.05,0.022,0.097,0.07


In [6]:
ORIGINS = pd.read_csv('../test_data/raw_data/all.csv', skipinitialspace=True, usecols=['origin'])['origin'].unique()
ORIGINS

array(['zbmath', 'genderize_r_authors', 'genderize_r_titles', 'pubmed',
       'wos'], dtype=object)

In [7]:
ORDERED_ERRORS = list(df.columns)
ORDERED_ERRORS

['errorCoded',
 'errorCodedWithoutNA',
 'errorGenderBias',
 'naCoded',
 'WeightedError']

In [8]:
df1 = compute_all_errors_without_tuning(origin='zbmath')
df1 = reduce_table(df1, by_index=True)
df1 = df1.round(3)

df1.style.background_gradient(cmap=cm)

,errorCoded,errorCodedWithoutNA,errorGenderBias,naCoded,WeightedError
NameAPI,0.063,0.018,0.018,0.046,0.027
NamSor,0.043,0.006,0.006,0.037,0.014
gender-guesser,0.066,0.003,0.003,0.063,0.016
Gender API,0.009,0.003,0.003,0.006,0.004
genderize.io,0.066,0.009,0.009,0.057,0.021


In [9]:
df2 = compute_all_errors_without_tuning(origin='genderize_r_authors')
df2 = reduce_table(df2, by_index=True)
df2 = df2.round(3)

df2.style.background_gradient(cmap=cm)

,errorCoded,errorCodedWithoutNA,errorGenderBias,naCoded,WeightedError
NameAPI,0.099,0.016,0.011,0.084,0.034
NamSor,0.029,0.012,-0.002,0.017,0.016
gender-guesser,0.08,0.005,0.005,0.075,0.021
Gender API,0.029,0.012,0.002,0.017,0.016
genderize.io,0.067,0.02,0.015,0.048,0.03


In [10]:
df3 = compute_all_errors_without_tuning(origin='genderize_r_titles')
df3 = reduce_table(df3, by_index=True)
df3 = df3.round(3)

df3.style.background_gradient(cmap=cm)

,errorCoded,errorCodedWithoutNA,errorGenderBias,naCoded,WeightedError
NameAPI,0.072,0.011,0.011,0.062,0.024
NamSor,0.04,0.022,0.022,0.019,0.025
gender-guesser,0.079,0.014,0.005,0.066,0.027
Gender API,0.034,0.017,0,0.017,0.021
genderize.io,0.081,0.034,0.029,0.049,0.043


In [11]:
df4 = compute_all_errors_without_tuning(origin='pubmed')
df4 = reduce_table(df4, by_index=True)
df4 = df4.round(3)

df4.style.background_gradient(cmap=cm)

,errorCoded,errorCodedWithoutNA,errorGenderBias,naCoded,WeightedError
NameAPI,0.103,0.021,-0.002,0.084,0.039
NamSor,0.07,0.024,0.001,0.047,0.033
gender-guesser,0.115,0.01,-0.002,0.106,0.033
Gender API,0.04,0.029,-0.008,0.011,0.032
genderize.io,0.07,0.023,0.008,0.047,0.033


In [12]:
df5 = compute_all_errors_without_tuning(origin='wos')
df5 = reduce_table(df5, by_index=True)
df5 = df5.round(3)

df5.style.background_gradient(cmap=cm)

,errorCoded,errorCodedWithoutNA,errorGenderBias,naCoded,WeightedError
NameAPI,0.283,0.057,0.003,0.24,0.113
NamSor,0.214,0.074,0.011,0.151,0.106
gender-guesser,0.37,0.054,0.005,0.334,0.14
Gender API,0.133,0.085,-0.019,0.052,0.095
genderize.io,0.23,0.087,0.036,0.156,0.12


In [13]:
dfs = pd.concat([df1,df2,df3,df4,df5], axis=1, keys=ORIGINS)
dfs 

zbmath                                              \
               errorCoded errorCodedWithoutNA errorGenderBias naCoded   
NameAPI             0.063               0.018           0.018   0.046   
NamSor              0.043               0.006           0.006   0.037   
gender-guesser      0.066               0.003           0.003   0.063   
Gender API          0.009               0.003           0.003   0.006   
genderize.io        0.066               0.009           0.009   0.057   

                             genderize_r_authors                      \
               WeightedError          errorCoded errorCodedWithoutNA   
NameAPI                0.027               0.099               0.016   
NamSor                 0.014               0.029               0.012   
gender-guesser         0.016               0.080               0.005   
Gender API             0.004               0.029               0.012   
genderize.io           0.021               0.067               0.020   

                                                          ...          pubmed  \
               errorGenderBias naCoded WeightedError      ...      errorCoded   
NameAPI                  0.011   0.084         0.034      ...           0.103   
NamSor                  -0.002   0.017         0.016      ...           0.070   
gender-guesser           0.005   0.075         0.021      ...           0.115   
Gender API               0.002   0.017         0.016      ...           0.040   
genderize.io             0.015   0.048         0.030      ...           0.070   

                                                                          \
               errorCodedWithoutNA errorGenderBias naCoded WeightedError   
NameAPI                      0.021          -0.002   0.084         0.039   
NamSor                       0.024           0.001   0.047         0.033   
gender-guesser               0.010          -0.002   0.106         0.033   
Gender API                   0.029          -0.008   0.011         0.032   
genderize.io                 0.023           0.008   0.047         0.033   

                      wos                                              \
               errorCoded errorCodedWithoutNA errorGenderBias naCoded   
NameAPI             0.283               0.057           0.003   0.240   
NamSor              0.214               0.074           0.011   0.151   
gender-guesser      0.370               0.054           0.005   0.334   
Gender API          0.133               0.085          -0.019   0.052   
genderize.io        0.230               0.087           0.036   0.156   

                              
               WeightedError  
NameAPI                0.113  
NamSor                 0.106  
gender-guesser         0.140  
Gender API             0.095  
genderize.io           0.120  

[5 rows x 25 columns]

In [14]:
dfs.columns = dfs.columns.swaplevel(0, 1)
dfs.sortlevel(0, axis=1, inplace=True)
dfs

/Users/helena.mihaljevic-br/.virtualenvs/name-based-inference/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  


WeightedError                                          \
               genderize_r_authors genderize_r_titles pubmed    wos zbmath   
NameAPI                      0.034              0.024  0.039  0.113  0.027   
NamSor                       0.016              0.025  0.033  0.106  0.014   
gender-guesser               0.021              0.027  0.033  0.140  0.016   
Gender API                   0.016              0.021  0.032  0.095  0.004   
genderize.io                 0.030              0.043  0.033  0.120  0.021   

                        errorCoded                                          \
               genderize_r_authors genderize_r_titles pubmed    wos zbmath   
NameAPI                      0.099              0.072  0.103  0.283  0.063   
NamSor                       0.029              0.040  0.070  0.214  0.043   
gender-guesser               0.080              0.079  0.115  0.370  0.066   
Gender API                   0.029              0.034  0.040  0.133  0.009   
genderize.io                 0.067              0.081  0.070  0.230  0.066   

                ...       errorGenderBias                                   \
                ...   genderize_r_authors genderize_r_titles pubmed    wos   
NameAPI         ...                 0.011              0.011 -0.002  0.003   
NamSor          ...                -0.002              0.022  0.001  0.011   
gender-guesser  ...                 0.005              0.005 -0.002  0.005   
Gender API      ...                 0.002              0.000 -0.008 -0.019   
genderize.io    ...                 0.015              0.029  0.008  0.036   

                                  naCoded                                   \
               zbmath genderize_r_authors genderize_r_titles pubmed    wos   
NameAPI         0.018               0.084              0.062  0.084  0.240   
NamSor          0.006               0.017              0.019  0.047  0.151   
gender-guesser  0.003               0.075              0.066  0.106  0.334   
Gender API      0.003               0.017              0.017  0.011  0.052   
genderize.io    0.009               0.048              0.049  0.047  0.156   

                       
               zbmath  
NameAPI         0.046  
NamSor          0.037  
gender-guesser  0.063  
Gender API      0.006  
genderize.io    0.057  

[5 rows x 25 columns]

In [15]:
# Reorder columns
dfs = dfs[dfs.columns.set_levels(ORDERED_ERRORS, level=0)]
dfs = dfs[dfs.columns.set_levels(ORIGINS, level=1)]

In [16]:
dfs.style.set_table_styles([dict(selector="th",props=[('max-width', '150px')])]).background_gradient(cmap=cm)